In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPolygon
from tqdm import tqdm
import gdown

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
file_id = "1U34r_SnEZNpIPXM-_lRDjlin3vbTN_sL"
url = f"https://drive.google.com/uc?export=download&id={file_id}"

# Download the file
gdown.download(url, "FD_SALAAN_2021.csv", quiet=False)

In [ ]:
base = pd.read_csv("FD_SALAAN_2021.csv", delimiter=";")

In [ ]:
base.head()